In [1]:
cd ..

c:\Users\timph\GitHub\ds4geo_MainProject


In [2]:
import geopandas as gpd 
import pandas as pd 
import matplotlib.pyplot as plt 
import rasterio
from rasterio.plot import show

from sentinelhub import SHConfig, MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, \
    DataCollection, bbox_to_dimensions, DownloadRequest,time_utils,SentinelHubBatch
import datetime
import numpy as np
import Processing as pc 

In [3]:
ski_areas = gpd.read_file('https://services3.arcgis.com/hG7UfxX49PQ8XkXh/arcgis/rest/services/URP_Schigebietsgrenzen/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [4]:
CLIENT_ID = '7d99e5b3-11f6-4b44-907c-b056294eee86'
CLIENT_SECRET = 'wV[i9n]PkAw>?Jy-e3IK]Fn.-^yX[t3wa<)8Ch7R'

In [5]:
config = SHConfig()
config.sh_client_id = CLIENT_ID
config.sh_client_secret = CLIENT_SECRET

In [6]:
evalscript_NDSI = """      

var NDSI = (B03 - B11) / (B03 + B11);
var NDVI = (B08 - B04) / (B08 + B04);
var gain = 2.5;

function si(a) {
    return (a>=0.4) ? 1 : (Math.abs(NDVI - 0.1) <= 0.025 ? 1 : 0);
}

function br(a) {
    return a>0.3;
}
   
var v = si(NDSI) && br(B03);

return (v==1) ? [0,0.7,1] : [B04, B03, B02].map(a => gain * a);
"""

In [7]:
serf_bbox, serf_size = pc.GetBBoxSize(ski_areas,'Komperdell')

In [8]:
pc.DownloadTimeseries(datetime.date(2019,11,1),
                      datetime.date(2019,12,31),
                      evalscript_NDSI,
                      serf_bbox,
                      serf_size,
                      config,
                      ski_area='Komperdell')